# Лабораторная работа 2 -- "Читаем хабр, используем Наташу, изучаем именованные сущности)

## Задание 1.

Открываем википедию *(за что мне это)*, выбираем статью не менее 3000 слов, определяем именованные сущности, которые фигурируют в тексте, определяем, какие границы этих сущностей хотим задать.

Текст большое, поэтому... Фух...

В качестве сущностей можем выделить: 
- локации -- Европа, Земля, Новая Зеландия
- временные периоды -- XIX век
- организации -- ВОЗ
- научные (и псевдонаучные, привет, википедия) определения -- суицид, парасуицид, псевдосуицид, скрытый суицид, попытка самоубийства, антисуицидальные факторы личности
- причины суицида *(сомнительно, конечно, что это можно прям выделять)* -- измены, семейные конфликты, неудачная любовь, госпитализм
- болезни -- ВИЧ-инфекция, нефрологические болезни, СПИД, бронхиальная астма, язвенная болезнь
- персоны -- А.Г. Амбрумова
- уровень и риск суицида -- как по странам, так и по местам, по полу и прочему, границу очертить тяжело
- способы суицида -- повешение, удушение, утопление
- наименование религиозных направлений -- католицизм, буддизм, джайнизм синтоизм + описание того, как в вере относятся к самоубийствам

На самом деле -- это не все. Мне кажется, здесь можно просто каждое слово вынести как именованную сущность и по сути прийти к морфологическому анализу :(

# Задание 2.

Извлекаем как можно больше именованных сущностей готовыми правилами

In [1]:
import requests

# Хорошо, что так можно сделать
# Жаль, что мне это не понравилось

response = requests.get(
    'https://ru.wikipedia.org/w/api.php',
    params={
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'explaintext': True,
        'titles': 'Самоубийство',
    }
).json()

f = open('analyze.txt', 'w')
f.write(next(iter(response['query']['pages'].values()))['extract'])
f.close()

In [2]:
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, PER, NamesExtractor, Doc 

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()

morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = open('analyze.txt').read()

doc = Doc(text)

# Рубрика "угадай, какую последовательность вызовов надо использовать, чтобы все это заработало"
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for it in doc.tokens:
    it.lemmatize(morph_vocab)

doc.parse_syntax(syntax_parser)
doc.sents[1].syntax.print()

          ┌► Чрезвычайно  advmod
        ┌►└─ сложный      amod
        │ ┌► этический    amod
        └─└─ вопрос       nsubj
┌─┌─────└─┌─ представляет 
│ │     ┌─└► отнесение    obj
│ │     │ ┌► к            case
│ │ ┌───└►└─ самоубийству nmod
│ │ │ ┌────► (            punct
│ │ │ │ ┌──► либо         cc
│ │ │ │ │ ┌► к            case
│ │ └►└─└─└─ убийству     parataxis
│ │     └──► )            punct
│ └────────► эвтаназии    obl
└──────────► .            punct


In [8]:
doc.tag_ner(ner_tagger)
doc.ner.print()
# аккуратно, тут будет много буков

Самоуби́йство, суици́д (от лат. sui caedere «убивать себя») — 
PER──────────                                                 
преднамеренное прекращение собственной жизни, как правило, 
самостоятельное и добровольное. Чрезвычайно сложный этический вопрос 
представляет отнесение к самоубийству (либо к убийству) эвтаназии.
Самопожертвование (например, на войне и в других экстремальных 
ситуациях) относят к проявлениям героизма и отличают от ординарного 
самоубийства и других разновидностей альтруистического самоубийства.
На территории Европы в XIX веке было зафиксировано увеличение числа 
              LOC───                                                
самоубийств, в XX веке эта тенденция усилилась. По данным Всемирной 
                                                          ORG───────
организации здравоохранения (ВОЗ), каждые 40 секунд кто-то из жителей 
─────────────────────────────────                                     
Земли уходит из жизни сознательно, кончая жизнь самоубийс

In [4]:
for span in doc.spans:
    span.normalize(morph_vocab)

# for span in doc.spans:
#     if span.type == PER:
#         print(span)

doc.spans[:50]

[DocSpan(stop=13, type='PER', text='Самоуби́йство', tokens=[...], normal='Самоуби́йство'),
 DocSpan(start=471, stop=477, type='LOC', text='Европы', tokens=[...], normal='Европа'),
 DocSpan(start=583, stop=626, type='ORG', text='Всемирной организации здравоохранения (ВОЗ)', tokens=[...], normal='Всемирная организация здравоохранения (ВОЗ)'),
 DocSpan(start=663, stop=668, type='LOC', text='Земли', tokens=[...], normal='Земля'),
 DocSpan(start=746, stop=749, type='ORG', text='ВОЗ', tokens=[...], normal='ВОЗ'),
 DocSpan(start=1030, stop=1047, type='LOC', text='Латинской Америке', tokens=[...], normal='Латинская Америка'),
 DocSpan(start=1119, stop=1130, type='LOC', text='Центральной', tokens=[...], normal='Центральная'),
 DocSpan(start=1133, stop=1148, type='LOC', text='Северной Европе', tokens=[...], normal='Северная Европа'),
 DocSpan(start=1150, stop=1166, type='LOC', text='Северной Америке', tokens=[...], normal='Северная Америка'),
 DocSpan(start=1168, stop=1186, type='LOC', text='Юго

### Легко увидеть, что...
В общем и целом мы очень хорошо справляемся с местами локаций. Шутка про то, что самоубийство -- это имя, вообще разрыв. Также прикольно, что у нас "Аффективные" -- это тоже персона, а СИЗО -- это организация... Но происходит это по понятным причинам, а, значит, можно предугадать такое поведение и просто хорошенько предобработать текст

## Правило 1
Далее первое правило нацелено на то, чтобы выделить в тексте какие-то заболевания, сопутствующие теме суицидологии. Я рассчитываю на то, что это позволит получить список "симптоматик" для более строгого надзора за людьми в группах риска

In [5]:
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, is_capitalized, dictionary
from yargy.pipelines import morph_pipeline

# Берем слова, которые должны описывать какой-то disease
DESIESE = morph_pipeline([
    'инфекция',
    'болезнь',
    'боль',
    'заболевание',
    'травма'
])

# Наименование болезни, инфекции, болей, которое скорее всего будет либо существительным, либо прилагательным (полным)
DESIESE_NAME = or_(
    gram("NOUN"),
    gram("ADJF"),
    # gram("ADJS")
)

# Предполагаем, что у нас либо сначала наименование, а потом что-то ужасное, либо наоборот
RULE = or_(
    rule(DESIESE, DESIESE_NAME),
    rule(DESIESE_NAME, DESIESE),
)

parser = Parser(RULE)
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['серьёзных', 'заболеваниях']
['душевную', 'боль']
['болезнь', 'близкого']
['соматическое', 'заболевание']
['психическими', 'травмами']
['хроническими', 'болями']
['хронические', 'заболевания']
['Соматические', 'болезни']
['соматическими', 'болезнями']
['хроническими', 'заболеваниями']
['и', 'травмы']
['онкологические', 'заболевания']
['непереносимые', 'боли']
['нефрологические', 'болезни']
['инфекцией', 'повышенный']
['соматических', 'болезней']
['хронические', 'заболевания']
['такие', 'заболевания']
['язвенная', 'болезнь']
['соматических', 'заболеваниях']
['психических', 'заболеваний']
['опасными', 'заболеваниями']
['душевной', 'боли']
['неизлечимые', 'болезни']
['серьёзного', 'заболевания']
['лечения', 'заболевания']
['неизлечимому', 'заболеванию']
['его', 'болезни']


## Результат:

Получилось не очень хорошо, но и не очень плохо. Мы не смогли найти ВИЧ-инфекцию и бронхиальную астму, однако нашли большой перечень хороших "звоночков". Но также имеются и проблемы: нашли сочетания по типу "его болезни" и "и травмы". С одной стороны, это ок, а с другой -- до полной автоматизации тут далеко.

# Правило 2
Сделаем вид, что нам очень лень самим читать текст и мы хотим узнать, кто что сделал во сколько раз чаще, чтобы привести эти в дипломчике. Такие сравнения, благо, часто носят одну и ту же форму, поэтому мы без проблем можем это попробовать сделать:

In [6]:
from yargy import Parser, rule, or_
from yargy.predicates import gram, is_capitalized, dictionary

# в
PREP = gram("PREP")

# 4
NUMB = or_(
    gram("NUMB"),
    gram("NUMR")
)

# раза
TIMES = morph_pipeline([
    'раза',
    'раз'
])

# чаще, реже
ADJ = or_(
    gram("ADJF"),
    gram("ADJS"),
    gram("COMP")
)

# умирают, убивают, режут, кушают
VERB = gram("VERB")

RULE = or_(
    rule(PREP, NUMB, TIMES, ADJ, VERB), # "в 10 раз чаще умирают"
    rule(PREP, NUMB, TIMES, ADJ, gram("NOUN")), # в 10 раз чаще мужчин
    rule(PREP, NUMB, TIMES), # "в 10 раз"
)

parser = Parser(RULE)
for match in parser.findall("Мужчины совершают самоубийство в 4.34 раза чаще, чем женщины (хотя женщины совершают в 4 раза больше попыток самоубийства)"):
    print([_.value for _ in match.tokens])

### И вот это уже незадача. Почему-то именно gram("NUMB") никак не может распознать число 4 или 4.34. Почему? Для меня загадка и интернет на этот счет молчит.

У pymorphy есть еще is_roman_number, но это нам как бы не подойдет... Очень странная и плохая дичь ._.

P.S. -- ROMN тоже не работает ;-;

# Правило 3

Ну что ж, после той неудачи, давайте попробуем найти что-нибудь

In [7]:
from yargy import Parser, rule, or_, not_
from yargy.predicates import gram, is_capitalized, dictionary, lte, gte, is_single


# Номер
NUMB_FIR = morph_pipeline([
    "номер",
    "#",
    "№"
])

# Числительное (а вот так работает))))
NUMB_SEC = and_(
    gte(1),
    lte(999)
)

# УК, РФ, ФЗ и прочее
LAW = and_(
    gram("NOUN"),
    is_single(),
    not_(gram("PREP")), # ну да, предлоги же -- это существительные...
    not_(gram("COMP")),
    not_(gram("CONJ")),
    not_(gram("NUMR")),
    or_(
        gram("nomn"), # именительный
        gram("accs"), # винительный
        gram("gent") # родительный
    )
)

RULE = or_(
    rule(NUMB_FIR, NUMB_SEC, "-", LAW), # № 100-ФЗ
    rule(NUMB_FIR, NUMB_SEC, LAW), # № 120 УК
    rule(NUMB_SEC, "-", LAW), # 120-ФЗ
    rule(NUMB_SEC, LAW, LAW), # 120 УК РФ, 120 ГК РФ
)

parser = Parser(RULE)
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['3', 'тысячи', 'человек']
['1', 'миллиона', 'человек']
['110', 'Уголовного', 'кодекса']
['120', '-', 'ФЗ']
['№', '139', '-', 'ФЗ']
['124', 'УК', 'РФ']


## Результат:
было бы проще и лучше воспользоваться регуляркой со словарем "УК", "ФЗ", "ГК", однако так как тут природа текста не очень профессиональная, она позволила также определить и "Уголовного кодекса", что круто, даже несмотря на то, что мне так и не удалось отрезать эти несчастные "3 тысячи человек" и "1 миллиона человек"

А еще забавно, что у нас "тысячи" -- это слово в единственном числе.